In [14]:
import tensorflow as tf
tf.enable_eager_execution()
import time
import pickle
from CleaningFunctions import *

In [15]:
from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

In [16]:
# get all segment file names and save in Filenames
folder = '/media/arash/My Passport/waymo'#'./waymo-od/data'
keyword = 'segment'
Filenames=scan(folder,keyword)

In [26]:
len(Filenames)

100

In [ ]:
### Extract segment files in Frames, each file consists of multiple frames

ObjectType = 2   #object codes, unknown:0, veh:1, ped:2, sign:3, bike:4   
count = len(Filenames)


for i in range(int(count/5)):     #read files three by three
    runtime0 = time.process_time()
    Frames = {}
    for file in Filenames[5*i:(5*i+5)]:
        SegID = getID(file)
        Segment = []
        dataset = tf.data.TFRecordDataset(file, compression_type='')
        for data in dataset:
            frame = open_dataset.Frame()
            frame.ParseFromString(bytearray(data.numpy()))
            Segment.append(frame)
        Frames[SegID] = Segment
        
    ObjData = MultiFrame(Frames,ObjectType)
    fileNumber = getID(file)[0:4]
    name = f"PedTables/File_{fileNumber}_Iterate_{5*i}:{5*i+5}.p"
    with open(name, 'wb') as fp:
        pickle.dump(ObjData, fp)
        
    runtime1 = time.process_time()
    print(f"File {i} Saved, Process Time: {runtime1 - runtime0}")
    del ObjData
    del Frames
  


File 0 Saved, Process Time: 5.736529739000002


In [2]:
## combining all the dics together
# get all names pf dict created and save in Filenames
folder = './PedTables'#'./waymo-od/data'
keyword = 'File'
Filenames=scan(folder,keyword)

In [3]:
## Open and merge all
AllPeds = {}
for i in range(len(Filenames)):
    with open(Filenames[i], 'rb') as fp:
        AllPeds.update(pickle.load(fp))

In [13]:
with open('./PedTables/AllPeds.p', 'wb') as fp:
    pickle.dump(AllPeds, fp)

In [10]:
PedIDs = list(AllPeds.keys())
AllPeds[PedIDs[1]]

,Timestamp,x,y,z,px_GF,py_GF,pz_GF,x_GF_SDC,y_GF_SDC,z_GF_SDC,...,Speed x,Speed y,accel x,accel y,Veh Count,Ped Count,Bike Count,Weather,Time of Day,Segment ID
0,1.550081e+15,14.414602,-6.235064,0.873832,-1557.301521,9912.500305,34.821055,-1545.183012,9922.485056,33.890,...,-1.163493,0.509982,-0.125217,0.107743,34.0,2.0,0.0,sunny,Day,0000_1083056852838271990_4080_000_4100_000
1,1.550081e+15,14.314980,-6.369815,0.871482,-1557.419510,9912.560931,34.821055,-1545.227580,9922.396715,33.890,...,-1.176015,0.520756,-0.125217,0.107743,34.0,2.0,0.0,sunny,Day,0000_1083056852838271990_4080_000_4100_000
2,1.550081e+15,14.211825,-6.505142,0.873410,-1557.537491,9912.621552,34.821055,-1545.274079,9922.306370,33.889,...,-1.188537,0.531530,-0.125217,0.107743,34.0,2.0,0.0,sunny,Day,0000_1083056852838271990_4080_000_4100_000
3,1.550081e+15,14.105416,-6.639873,0.875120,-1557.655487,9912.682181,34.821055,-1545.322268,9922.213059,33.888,...,-1.201059,0.542305,-0.125217,0.107743,34.0,2.0,0.0,sunny,Day,0000_1083056852838271990_4080_000_4100_000
4,1.550081e+15,13.994459,-6.774544,0.871275,-1557.773469,9912.742803,34.821055,-1545.371961,9922.114396,33.886,...,-1.213580,0.553079,-0.125217,0.107743,34.0,2.0,0.0,sunny,Day,0000_1083056852838271990_4080_000_4100_000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,1.550081e+15,-18.506558,-28.271230,2.719444,-1576.417391,9921.834126,35.727723,-1561.456686,9891.450017,34.267,...,-1.132828,0.785109,0.206787,0.144434,40.0,2.0,0.0,sunny,Day,0000_1083056852838271990_4080_000_4100_000
148,1.550081e+15,-18.887250,-28.522420,2.657943,-1576.530827,9921.912817,35.721053,-1561.726680,9890.990134,34.326,...,-1.112149,0.799553,0.206787,0.144434,36.0,2.0,0.0,sunny,Day,0000_1083056852838271990_4080_000_4100_000
149,1.550081e+15,-19.268808,-28.780720,2.621515,-1576.644208,9921.991469,35.714386,-1562.001161,9890.522600,34.380,...,-1.091470,0.813996,0.206787,0.144434,36.0,1.0,0.0,sunny,Day,0000_1083056852838271990_4080_000_4100_000
150,1.550081e+15,-19.664304,-29.029861,2.648103,-1576.757587,9922.070121,35.707720,-1562.280749,9890.051005,34.432,...,-1.070792,0.828439,0.206787,0.144434,35.0,1.0,0.0,sunny,Day,0000_1083056852838271990_4080_000_4100_000


In [5]:
#adjust col names 
# def dummychangename(df):
#     df.columns = ['Timestamp', 'x', 'y', 'z',
#                     'px_GF', 'py_GF', 'pz_GF',
#                     'x_GF_SDC', 'y_GF_SDC', 'z_GF_SDC',
#                     'Width', 'Length', 'Height',
#                     'Heading','Speed x', 'Speed y',
#                     'accel x', 'accel y','Veh Count','Ped Count',
#                   'Bike Count','Weather','Time of Day', 'Segment ID']
#     return df

In [6]:
# d = {k : dummychangename(v) for k, v in AllPeds.items()}
